# Модель предсказания UpLift для компании "Лента"

In [1]:
! pip install CatBoost
! pip install LightGBM

In [2]:
! pip install -U pandasql

Requirement already up-to-date: pandasql in /anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages (0.7.3)


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
#import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy import stats as st
import matplotlib.pyplot as plt
#import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=3,suppress=True)
import warnings
warnings.simplefilter("ignore")
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
import time
#from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
import datetime

In [4]:
#import pandasql as ps

## Подготовка данных

In [5]:
checks = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_checks.csv')

In [6]:
hierarchy = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_hierarchy.csv')

In [7]:
uplift = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_uplift.csv')

In [8]:
offers = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210521_offers.csv')

Создадим функцию для перевода данных из формата int в формат даты.

In [9]:
def datification(weird_date):
    new_date = datetime.datetime.strptime(weird_date, '%Y%m%d')
    return new_date

In [10]:
offers['start_date'] = offers['start_date'].astype(str)
offers['start_date'] = offers['start_date'].apply(datification)
offers['end_date'] = offers['end_date'].astype(str)
offers['end_date'] = offers['end_date'].apply(datification)

In [11]:
offers.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,2020-01-02,2020-01-08,train
2,Facades,F9,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,train
3,Facades,F45,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,train
4,Facades,F191,e87788b585ead792a4d40f7de18f2f76,2020-11-12,2020-11-18,test


In [12]:
checks['day'] = checks['day'].astype(str)
checks['day'] = checks['day'].apply(datification)

In [13]:
checks.head()

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name
0,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b17c870027bb4a22e3aedb971bc00def,b3deb0286313f0b888c0eac49580cc23,1,5.000,133.400,249.450,152f1b77a32508570e2745daf9ce7aec
1,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,152f1b77a32508570e2745daf9ce7aec
2,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,5,2.000,2.340,7.580,152f1b77a32508570e2745daf9ce7aec
3,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b25e0d42165d246d23cbf4d3efcdf583,7215ee9c7d9dc229d2921a40e899ec5f,4,1.000,23.590,39.990,152f1b77a32508570e2745daf9ce7aec
4,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,a537d2eef30d47f9b11d655421ca93fb,05c93d4fa9bdbe70b5d2204ce9548ca9,3,1.000,24.020,39.990,152f1b77a32508570e2745daf9ce7aec


In [14]:
checks['selling_price_per_unit'] = checks['selling_price'] / checks['num_sales']

In [15]:
checks['supplier_price_per_unit'] = checks['supplier_price'] / checks['num_sales']

In [16]:
checks['promo_id'].value_counts().sort_values(ascending = False)

7215ee9c7d9dc229d2921a40e899ec5f    10881653
c4c67f412855bab90c566139cb2b3d83      182037
47b5739208ae99fe23f06fedb3ee8ea7       74030
d9ca24d75e5cecc5a01c452cc8b8288e       72601
7071bf06286a55deac8fa9619ae3517a       71467
                                      ...   
becf1a35d76bda0d7ad9a43fa129e183           1
c231afaceff2d44d6ed55ffe556ec99d           1
2d6802b6509c34e22157ced9d4060861           1
a9be514248060d79f26b58de651e56d6           1
88d0db293c68bd4bd93a7aa63852b91c           1
Name: promo_id, Length: 176939, dtype: int64

Больше всего товаров продаётся без акции. Заменим это значение promo_id на no_offers

In [17]:
checks['promo_id'] = checks['promo_id'].mask(checks['promo_id'] == '7215ee9c7d9dc229d2921a40e899ec5f', 'no_offers')

In [18]:
checks['promo_id'].value_counts().sort_values(ascending = False)

no_offers                           10881653
c4c67f412855bab90c566139cb2b3d83      182037
47b5739208ae99fe23f06fedb3ee8ea7       74030
d9ca24d75e5cecc5a01c452cc8b8288e       72601
7071bf06286a55deac8fa9619ae3517a       71467
                                      ...   
daf4b20ce064b410d0a3ada2bb413208           1
becf1a35d76bda0d7ad9a43fa129e183           1
c231afaceff2d44d6ed55ffe556ec99d           1
2d6802b6509c34e22157ced9d4060861           1
7783760c4f4a5e4e4fa4077129ba1b6f           1
Name: promo_id, Length: 176939, dtype: int64

Добавим к таблице checks данные по иерархии товаров.

In [19]:
checks_merged1 = checks.merge(hierarchy, how='outer', on = ['sku'])

In [20]:
offers['duration'] = offers['end_date'] - offers['start_date'] + pd.to_timedelta('1 day')

Для определения предакционного периода введём дату pre_start_datepre_start_date

In [21]:
offers['pre_start_date'] = offers['start_date'] - offers['duration']

In [22]:
checks_with_offers = checks_merged1.merge(offers, how='outer', on = ['sku'])

checks_with_offers_train = checks_with_offers.query('train_test_group in ("train")')

checks_with_offers_train

checks_with_offers_train.to_csv('checks_with_offers_train.csv')

In [23]:
checks_with_offers_action = checks_with_offers.query('day <= end_date and day >= start_date')

In [24]:
checks_with_offers_pred_action = checks_with_offers.query('day < start_date and day >= pre_start_date')

checks_with_offers_action.to_csv('checks_with_offers_action.csv')

checks_with_offers_pred_action.to_csv('checks_with_offers_pred_action.csv')

In [25]:
checks_with_offers_action

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,Promo_type,Offer_ID,start_date,end_date,train_test_group,duration,pre_start_date
1479,f5883bc2315398454cef231404da81ee,2020-03-27,d39934ce111a864abf40391f3da9cdf5,6b4d6d3a10deb5788eafca73c0e795fd,13:01:11,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,46,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1481,62823feeae170ce71a3787c203d8a718,2020-03-27,d39934ce111a864abf40391f3da9cdf5,4cec157ad32ea17bf0a3e4178019b990,15:15:53,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,1,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1483,073aa4b0edc4617c62c302aaf791a7cb,2020-03-27,d39934ce111a864abf40391f3da9cdf5,502843a48b0fc49f4acc134a7f0c03cd,08:00:35,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,14,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1485,9925f967444c9cee55518b25ee7a116a,2020-03-27,d39934ce111a864abf40391f3da9cdf5,d973669af03fca24d481b7b28d0256e9,20:51:32,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,24,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1487,cd6e626e2303c8c34c2b411d974a066a,2020-03-27,d39934ce111a864abf40391f3da9cdf5,3ba293abef2c1328081ccb6bc88524df,16:12:32,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,1,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34328625,8ad501db6e0f885585b2e0dfa0e68d13,2020-08-19,d33f67c5652acb3fac99b9dc16bea0c1,99281b4fedff4c7d1f393ab80736b90d,11:20:05,80047755fdd5c6459ed2c4f734c190e4,792172eaeacc7979a5d0f50359d13322,12,1.000,206.140,298.430,7e35e74e610188414ad24235dd787c78,298.430,206.140,44a7b825383402ed937a15fb8a5bb12a,2838023a778dfaecdc212708f721b788,2c26f9a59b0ba61233e6fc0af8e47f14,e2c7a001428c33b71d08b54f254e0997,Seasonal,506,2020-07-28,2020-09-01,train,36 days,2020-06-22
34330360,fb704ef73f277eeff1154e7a8112941c,2020-06-28,29549a71a57f587d88209b9c1f1b7999,b5b9528029645c816ec2bc6858657855,15:21:39,9fe155a9e4f9402ba8728916a35a162e,883319be348e4a942d64134fbbbaf055,49,1.000,125.400,178.820,152f1b77a32508570e2745daf9ce7aec,178.820,125.400,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,f50c7035e532c49a0f6993d988e2e843,c134be80fec34c5b05f7464154900af8,Biweekly,368,2020-06-25,2020-07-08,train,14 days,2020-06-11
34330977,f40fefad0e55aa5c99a7e95553cdafef,2019-10-07,147768d3955e38c4e662c0a95d807abc,195b515ef0ea2719ebefd618b102d96c,13:26:34,275e38c99e9868037585564842839a8e,019043b5539be61b162515a5e0d9a5a7,11,1.000,455.930,798.630,152f1b77a32508570e2745daf9ce7aec,798.630,455.930,44a7b825383402ed937a15fb8a5bb12a,c0c7c76d30bd3dcaefc96f40275bdc0a,3202111cf90e7c816a472aaceb72b0df,d854234cf1f26b77b259a8511912040b,S

In [26]:
checks_with_offers_pred_action

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,Promo_type,Offer_ID,start_date,end_date,train_test_group,duration,pre_start_date
965,1aa391faa9bac6134a0aa1aa6aa51fd8,2020-03-07,8c5781246fcb0296e1f11cd4e2eee68d,5c972464239bd4147d82e14ff74a93ef,17:38:37,b9ef00e971d0fb459a765708efba70c0,no_offers,12,1.000,100.640,188.890,7e35e74e610188414ad24235dd787c78,188.890,100.640,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
967,677635a094c51b57726420f4ad6ee3e9,2020-03-07,adeb12111c95807557664efbf3b746ae,9ec1b02bc03e904bc031aadeafeb624c,08:03:23,b9ef00e971d0fb459a765708efba70c0,no_offers,13,1.000,104.310,199.390,7e35e74e610188414ad24235dd787c78,199.390,104.310,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
969,06e50e2e042604a4c15f1d10d1d84911,2020-03-07,a410115ed452a9213424d92cacd2a083,a3bfb5ca27358cd89eefef669443906a,18:42:59,b9ef00e971d0fb459a765708efba70c0,no_offers,2,1.000,114.910,194.990,152f1b77a32508570e2745daf9ce7aec,194.990,114.910,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
971,7275f3537f8a51ceeb5c66daac3786da,2020-03-07,1640f59af9fc45f93a72652de254d387,2bb71bd0be4750323de82b8b3f281f52,14:02:17,b9ef00e971d0fb459a765708efba70c0,no_offers,12,1.000,138.250,183.020,152f1b77a32508570e2745daf9ce7aec,183.020,138.250,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1179,e18655bda57e4041819e83c1c6993667,2020-03-07,f9d3742964e8d35a46da512cf46fbeb3,d7cf11065fa72ac1f963b77708fb30f0,19:52:23,b9ef00e971d0fb459a765708efba70c0,no_offers,12,1.000,96.770,195.090,7e35e74e610188414ad24235dd787c78,195.090,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34313596,7553ca3847970838d2eb0c8e7482b574,2020-07-13,c0279f73075a52e1a7dea35065bc8c80,071796c257614814b2a3628df9c877ae,11:37:38,62237c588c8b170a1aa6388b5479ce60,no_offers,14,1.000,74.920,142.410,152f1b77a32508570e2745daf9ce7aec,142.410,74.920,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,c5d215777c229704a7862de577d40a73,31873776d238ef88ce7f69c799f7edfd,Seasonal,510,2020-07-28,2020-09-01,train,36 days,2020-06-22
34313952,bea55df34ca1452092fb3815f1a5f7e5,2020-06-07,898dc2c947cee718e4afd7dfcb2f1a09,92a98b1e61b9ad123d3acf3d9b393874,14:58:16,26f6fb076274b4416f995fe47547e803,a5529da46e1cea07199b3a4a906a8421,1,1.000,5561.190,13999.000,7e35e74e610188414ad24235dd787c78,13999.000,5561.190,44a7b825383402ed937a15fb8a5bb12a,66f041e16a60928b05a7e228a89c3799,1ce4fe042832e6bd7d06697a43055373,662e87c6f8d64f5261607715aff07b66,Seasonal,523,2020-06-29,2020-07-27,train,29 days,2020-05-31
34313957,c339acf196f01f1b4b1639b7fe7a7790,2020-06-08,e90b327c1ceca11f1a44a8bcf58af34c,87a576d6a59bd60459d87b672cadd045,14:33:24,26f6fb076274b4416f995fe47547e803,a5529da46e1cea07199b3a4a906a8421,1,1.000,6003.450,13999.000,7e35e74e610188414ad24235dd787c78,13999.000,6003.450,44a7b825383402ed937a15fb8a5bb12a,66f041e16a60928b05a7e228a89c3799,1ce4fe042832e6bd7d06697a43055373,662e87c6f8d64f5261607715aff07b66,Seasonal,523,2020-06-29,2020-07-27,train,29 days,2020-05-31
34318765,463a85bdd0ea19a697c62dba8141fb90,2020-06-24,95b09698fda1f

In [27]:
offers

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,duration,pre_start_date
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,7 days,2020-02-06
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,2020-01-02,2020-01-08,train,7 days,2019-12-26
2,Facades,F9,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,train,7 days,2020-01-09
3,Facades,F45,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,train,7 days,2020-03-12
4,Facades,F191,e87788b585ead792a4d40f7de18f2f76,2020-11-12,2020-11-18,test,7 days,2020-11-05
...,...,...,...,...,...,...,...,...
9771,Biweekly,683,0f368be7897dd1531e43dc37115eeaf0,2019-11-14,2019-11-27,train,14 days,2019-10-31
9772,Biweekly,691,47c28bf957fe120f2d016e46911e5bdb,2019-11-14,2019-11-27,train,14 days,2019-10-31
9773,Biweekly,683,00432d021eca6059fee2ed69a732c2b2,2019-11-14,2019-11-27,train,14 days,2019-10-31
9774,Biweekly,690,40d78b3648a4ea15f300070a0ffd9752,2019-11-14,2019-11-27,train,14 days,2019-10-31


In [28]:
checks_with_offers_action_pivot_price = checks_with_offers_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date', 'train_test_group'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_offers_action_pivot_price.columns = ['mean_selling_price_per_unit_action', 'mean_supplier_price_per_unit_action']
checks_with_offers_action_pivot_price = checks_with_offers_action_pivot_price.reset_index()
checks_with_offers_action_pivot_price

,Offer_ID,sku,region_name,Promo_type,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.290,28.623
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,48.832,28.504
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.346,31.137
...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,115.438,110.175
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,71.952,48.791
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,81.594,38.961
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,29.943,22.317


In [29]:
checks_with_offers_action_pivot_amount = checks_with_offers_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date', 'train_test_group'], values=['num_sales', 'supplier_price', 'selling_price'] , aggfunc=['sum'])
checks_with_offers_action_pivot_amount.columns = ['num_sales_action', 'selling_price_action', 'supplier_price_action']
checks_with_offers_action_pivot_amount = checks_with_offers_action_pivot_amount.reset_index()
checks_with_offers_action_pivot_amount

,Offer_ID,sku,region_name,Promo_type,start_date,train_test_group,num_sales_action,selling_price_action,supplier_price_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,95.000,4684.180,2715.750
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,103.000,5036.900,2936.440
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,289.000,14265.200,8327.790
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,521.000,25722.260,14813.040
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,111.000,5477.810,3457.240
...,...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,3299.344,380506.710,363546.990
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,773.460,55035.570,37949.550
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,319.296,25920.530,12437.100
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,2511.510,75197.620,56061.950


In [30]:
checks_with_offers_pred_action_pivot_price = checks_with_offers_pred_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date',  'train_test_group'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_offers_pred_action_pivot_price.columns = ['mean_selling_price_per_unit_without_action', 'mean_supplier_price_per_unit_without_action']
checks_with_offers_pred_action_pivot_price = checks_with_offers_pred_action_pivot_price.reset_index()
checks_with_offers_pred_action_pivot_price

,Offer_ID,sku,region_name,Promo_type,start_date,train_test_group,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,77.996,29.990
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,74.865,28.818
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,82.490,30.930
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,83.990,30.983
...,...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,141.456,105.032
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,74.711,47.297
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,83.753,37.983
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,38.316,26.490


In [31]:
checks_with_offers_pred_action_pivot_amount = checks_with_offers_pred_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date',  'train_test_group'], values=['num_sales', 'supplier_price', 'selling_price'] , aggfunc=['sum'])
checks_with_offers_pred_action_pivot_amount.columns = ['num_sales_without_action', 'selling_price_without_action', 'supplier_price_without_action']
checks_with_offers_pred_action_pivot_amount = checks_with_offers_pred_action_pivot_amount.reset_index()
checks_with_offers_pred_action_pivot_amount

,Offer_ID,sku,region_name,Promo_type,start_date,train_test_group,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,20.000,1562.700,598.590
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,34.000,2537.990,986.540
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,2.000,164.980,61.860
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,1.000,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,5.000,419.950,154.860
...,...,...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,506.986,71652.860,53380.760
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,503.724,37428.130,23808.970
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,321.260,26918.890,12243.790
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,685.386,26440.340,18299.620


In [32]:
checks_pred_action_merged = checks_with_offers_pred_action_pivot_price.merge(checks_with_offers_pred_action_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'region_name', 'start_date',  'train_test_group'])
checks_pred_action_merged

,Offer_ID,sku,region_name,Promo_type_x,start_date,train_test_group,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,77.996,29.990,Biweekly,20.000,1562.700,598.590
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,74.865,28.818,Biweekly,34.000,2537.990,986.540
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,82.490,30.930,Biweekly,2.000,164.980,61.860
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,83.990,30.930,Biweekly,1.000,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,83.990,30.983,Biweekly,5.000,419.950,154.860
...,...,...,...,...,...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,141.456,105.032,Facades,506.986,71652.860,53380.760
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,74.711,47.297,Facades,503.724,37428.130,23808.970
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,83.753,37.983,Facades,321.260,26918.890,12243.790
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,38.316,26.490,Facades,685.386,26440.340,18299.620


In [33]:
checks_with_action_merged = checks_with_offers_action_pivot_price.merge(checks_with_offers_action_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'region_name', 'start_date',  'train_test_group'])
checks_with_action_merged

,Offer_ID,sku,region_name,Promo_type_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y,num_sales_action,selling_price_action,supplier_price_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.290,28.623,Biweekly,95.000,4684.180,2715.750
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,48.832,28.504,Biweekly,103.000,5036.900,2936.440
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.346,31.137,Biweekly,111.000,5477.810,3457.240
...,...,...,...,...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,115.438,110.175,Facades,3299.344,380506.710,363546.990
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,71.952,48.791,Facades,773.460,55035.570,37949.550
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,81.594,38.961,Facades,319.296,25920.530,12437.100
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,29.943,22.317,Facades,2511.510,75197.620,56061.950


In [34]:
checks_offers_merged = checks_with_action_merged.merge(checks_pred_action_merged, how='outer', on = ['Offer_ID', 'sku', 'region_name', 'start_date',  'train_test_group'])
checks_offers_merged

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.290,28.623,Biweekly,95.000,4684.180,2715.750,NaN,nan,nan,NaN,nan,nan,nan
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,48.832,28.504,Biweekly,103.000,5036.900,2936.440,NaN,nan,nan,NaN,nan,nan,nan
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,F5,edbe917ef00874205ad893c3931ea9e0,7e35e74e610188414ad24235dd787c78,NaN,2020-01-09,train,nan,nan,NaN,nan,nan,nan,Facades,101.990,95.600,Facades,1.000,101.990,95.600
14770,F50,f3f807ff0fe767b94827bc9d8367d324,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,train,nan,nan,NaN,nan,nan,nan,Facades,188.990,95.240,Facades,1.000,188.990,95.240
14771,F50,fe1d8a29a7ed76cd79f0a520ed18e0e6,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,train,nan,nan,NaN,nan,nan,nan,Facades,188.410,101.300,Facades,1.000,188.410,101.300
14772,F53,6b046ae5ddceb4ca2dca623fbc9011cf,7e35e74e610188414ad24235dd787c78,NaN,2020-04-02,train,nan,nan,NaN,nan,nan,nan,Facades,74.890,57.280,Facades,2.000,149.780,114.560


checks_offers_merged_without_null = checks_offers_merged.dropna()
checks_offers_merged_without_null.head()

checks_offers_merged_without_null.info()

del checks_offers_merged_without_null['Promo_type_y_x']
del checks_offers_merged_without_null['Promo_type_y_y']

In [35]:
checks_offers_train = checks_offers_merged.query('train_test_group in ("train")')
checks_offers_train

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.290,28.623,Biweekly,95.000,4684.180,2715.750,NaN,nan,nan,NaN,nan,nan,nan
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,48.832,28.504,Biweekly,103.000,5036.900,2936.440,NaN,nan,nan,NaN,nan,nan,nan
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,F5,edbe917ef00874205ad893c3931ea9e0,7e35e74e610188414ad24235dd787c78,NaN,2020-01-09,train,nan,nan,NaN,nan,nan,nan,Facades,101.990,95.600,Facades,1.000,101.990,95.600
14770,F50,f3f807ff0fe767b94827bc9d8367d324,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,train,nan,nan,NaN,nan,nan,nan,Facades,188.990,95.240,Facades,1.000,188.990,95.240
14771,F50,fe1d8a29a7ed76cd79f0a520ed18e0e6,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,train,nan,nan,NaN,nan,nan,nan,Facades,188.410,101.300,Facades,1.000,188.410,101.300
14772,F53,6b046ae5ddceb4ca2dca623fbc9011cf,7e35e74e610188414ad24235dd787c78,NaN,2020-04-02,train,nan,nan,NaN,nan,nan,nan,Facades,74.890,57.280,Facades,2.000,149.780,114.560


In [36]:
checks_offers_train['uplift'] = checks_offers_train['mean_selling_price_per_unit_action'] / checks_offers_train['mean_selling_price_per_unit_without_action'] - 1
checks_offers_train

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.290,28.623,Biweekly,95.000,4684.180,2715.750,NaN,nan,nan,NaN,nan,nan,nan,nan
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,48.832,28.504,Biweekly,103.000,5036.900,2936.440,NaN,nan,nan,NaN,nan,nan,nan,nan
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,-0.367
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,-0.341
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860,-0.402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,F5,edbe917ef00874205ad893c3931ea9e0,7e35e74e610188414ad24235dd787c78,NaN,2020-01-09,train,nan,nan,NaN,nan,nan,nan,Facades,101.990,95.600,Facades,1.000,101.990,95.600,nan
14770,F50,f3f807ff0fe767b94827bc9d8367d324,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,train,nan,nan,NaN,nan,nan,nan,Facades,188.990,95.240,Facades,1.000,188.990,95.240,nan
14771,F50,fe1d8a29a7ed76cd79f0a520ed18e0e6,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,train,nan,nan,NaN,nan,nan,nan,Facades,188.410,101.300,Facades,1.000,188.410,101.300,nan
14772,F53,6b046ae5ddceb4ca2dca623fbc9011cf,7e35e74e610188414ad24235dd787c78,NaN,2020-04-02,train,nan,nan,NaN,nan,nan,nan,Facades,74.890,57.280,Facades,2.000,149.780,114.560,nan


In [37]:
checks_offers_train = checks_offers_train.dropna()
checks_offers_train

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,-0.367
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,-0.341
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860,-0.402
5,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.387,31.173,Biweekly,174.000,8592.680,5419.670,Biweekly,83.990,30.930,Biweekly,1.000,83.990,30.930,-0.412
6,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.369,31.012,Biweekly,144.000,7108.950,4464.550,Biweekly,83.990,30.983,Biweekly,5.000,419.950,154.860,-0.412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,115.438,110.175,Facades,3299.344,380506.710,363546.990,Facades,141.456,105.032,Facades,506.986,71652.860,53380.760,-0.184
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,71.952,48.791,Facades,773.460,55035.570,37949.550,Facades,74.711,47.297,Facades,503.724,37428.130,23808.970,-0.037
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,81.594,38.961,Facades,319.296,25920.530,12437.100,Facades,83.753,37.983,Facades,321.260,26918.890,12243.790,-0.026
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,29.943,22.317,Facades,2511.510,75197.620,56061.950,Facades,38.316,26.490,Facades,685.386,26440.340,18299.620,-0.219


In [38]:
train = checks_offers_train.merge(hierarchy, how='inner', on = ['sku'])
train

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,-0.367,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,-0.341,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
2,F137,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,2020-08-27,train,49.380,30.773,Facades,113.000,5579.960,3459.870,Facades,74.404,29.826,Facades,11.000,794.580,328.720,-0.336,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
3,F137,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Facades,2020-08-27,train,49.368,29.893,Facades,126.000,6220.320,3755.490,Facades,83.953,29.529,Facades,5.000,416.240,148.760,-0.412,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
4,F287,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,2019-11-14,train,44.465,30.653,Facades,135.000,5994.560,4136.210,Facades,73.687,30.839,Facades,7.000,583.410,216.760,-0.397,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,F97,9794c588b279cd37153e81d9109acf70,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,44.027,31.706,Facades,25.000,1100.870,783.020,Facades,61.590,27.140,Facades,4.000,246.360,108.560,-0.285,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12505,F97,aa743eaaf303d1ded42c543797089f35,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,44.082,28.774,Facades,74.000,3262.260,2122.640,Facades,70.291,33.871,Facades,23.000,1656.810,755.510,-0.373,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12506,F97,aa743eaaf303d1ded42c543797089f35,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,44.080,27.922,Facades,48.000,2115.650,1338.390,Facades,75.270,31.369,Facades,8.000,602.160,250.950,-0.414,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12507,F97,d9b38fad6052a1c1045b18b67f34978b,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,273.589,163.764,Facades,21.000,5745.370,3439.040,Facades,383.010,180.750,Facades,3.000,1149.030,542.250,-0.286,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,7599d4e0d9b5eae4fecdc440358604b1


In [63]:
train['margin'] = train['mean_selling_price_per_unit_without_action'] / train['mean_supplier_price_per_unit_without_action'] - 1
train

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,train_test_group,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,margin
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,train,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,-0.367,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76,1.601
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,train,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,-0.341,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76,1.598
2,F137,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,2020-08-27,train,49.380,30.773,Facades,113.000,5579.960,3459.870,Facades,74.404,29.826,Facades,11.000,794.580,328.720,-0.336,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76,1.495
3,F137,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Facades,2020-08-27,train,49.368,29.893,Facades,126.000,6220.320,3755.490,Facades,83.953,29.529,Facades,5.000,416.240,148.760,-0.412,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76,1.843
4,F287,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,2019-11-14,train,44.465,30.653,Facades,135.000,5994.560,4136.210,Facades,73.687,30.839,Facades,7.000,583.410,216.760,-0.397,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76,1.389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,F97,9794c588b279cd37153e81d9109acf70,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,44.027,31.706,Facades,25.000,1100.870,783.020,Facades,61.590,27.140,Facades,4.000,246.360,108.560,-0.285,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa,1.269
12505,F97,aa743eaaf303d1ded42c543797089f35,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,44.082,28.774,Facades,74.000,3262.260,2122.640,Facades,70.291,33.871,Facades,23.000,1656.810,755.510,-0.373,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa,1.075
12506,F97,aa743eaaf303d1ded42c543797089f35,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,train,44.080,27.922,Facades,48.000,2115.650,1338.390,Facades,75.270,31.369,Facades,8.000,602.160,250.950,-0.414,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa,1.400
12507,F97,d9b38fad6052a1c1045b18b67f34978b,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,train,273.589,163.764,Facades,21.000,5745.370,3439.040,Facades,383.010,180.750,Facades,3.000,1149.030,542.250,-0.286,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,7599d4e0d9b5eae4fecdc440358604b1,1.119


В полученных данных не будет данных для тестовой выборки, т.к. у нас нет данных за 4 квартал 2020, поэтому отдельно займёмся подготовкой признаков для тестовой выборки. 

In [39]:
checks_with_offers = checks_merged1.merge(offers, how='outer', on = ['sku'])

In [40]:
checks_with_offers = checks_with_offers[checks_with_offers['Offer_ID'].isna() != True]

In [41]:
checks_with_offers_test = checks_with_offers.query('train_test_group in ("test")')

In [49]:
checks_with_offers_test

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,Promo_type,Offer_ID,start_date,end_date,train_test_group,duration,pre_start_date
749713,0c75c5e1422c6a716fde39ce44f1f8ef,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,77c585133342e38989226fbcbc381eb3,08:17:49,1cbaaeb2212bbbe9d458ca9b81d74db7,86b055ff16c357a59decc2d7d32f2cc7,7,2.856,108.900,55.680,152f1b77a32508570e2745daf9ce7aec,19.496,38.130,cc9284597026dcc021735c0970ca0c93,6f4922f45568161a8cdf4ad2299f6d23,645098b086d2f9e1e0e939c27f9f2d6f,d66a5517fea10ab5d81ee32912ef07e6,Billboards,B94,2020-11-12,2020-11-18,test,7 days,2020-11-05
749715,0c75c5e1422c6a716fde39ce44f1f8ef,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,77c585133342e38989226fbcbc381eb3,08:17:49,1cbaaeb2212bbbe9d458ca9b81d74db7,86b055ff16c357a59decc2d7d32f2cc7,7,2.856,108.900,55.680,152f1b77a32508570e2745daf9ce7aec,19.496,38.130,cc9284597026dcc021735c0970ca0c93,6f4922f45568161a8cdf4ad2299f6d23,645098b086d2f9e1e0e939c27f9f2d6f,d66a5517fea10ab5d81ee32912ef07e6,Facades,F176,2020-10-22,2020-10-28,test,7 days,2020-10-15
749716,0c75c5e1422c6a716fde39ce44f1f8ef,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,77c585133342e38989226fbcbc381eb3,08:17:49,1cbaaeb2212bbbe9d458ca9b81d74db7,86b055ff16c357a59decc2d7d32f2cc7,7,2.856,108.900,55.680,152f1b77a32508570e2745daf9ce7aec,19.496,38.130,cc9284597026dcc021735c0970ca0c93,6f4922f45568161a8cdf4ad2299f6d23,645098b086d2f9e1e0e939c27f9f2d6f,d66a5517fea10ab5d81ee32912ef07e6,Billboards,B102,2020-12-17,2020-12-23,test,7 days,2020-12-10
749719,0c75c5e1422c6a716fde39ce44f1f8ef,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,77c585133342e38989226fbcbc381eb3,08:17:49,1cbaaeb2212bbbe9d458ca9b81d74db7,86b055ff16c357a59decc2d7d32f2cc7,2,3.276,124.910,63.870,152f1b77a32508570e2745daf9ce7aec,19.496,38.129,cc9284597026dcc021735c0970ca0c93,6f4922f45568161a8cdf4ad2299f6d23,645098b086d2f9e1e0e939c27f9f2d6f,d66a5517fea10ab5d81ee32912ef07e6,Billboards,B94,2020-11-12,2020-11-18,test,7 days,2020-11-05
749721,0c75c5e1422c6a716fde39ce44f1f8ef,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,77c585133342e38989226fbcbc381eb3,08:17:49,1cbaaeb2212bbbe9d458ca9b81d74db7,86b055ff16c357a59decc2d7d32f2cc7,2,3.276,124.910,63.870,152f1b77a32508570e2745daf9ce7aec,19.496,38.129,cc9284597026dcc021735c0970ca0c93,6f4922f45568161a8cdf4ad2299f6d23,645098b086d2f9e1e0e939c27f9f2d6f,d66a5517fea10ab5d81ee32912ef07e6,Facades,F176,2020-10-22,2020-10-28,test,7 days,2020-10-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34329949,0132c6d2169585f82c568e79a66368b2,2020-09-13,e90b327c1ceca11f1a44a8bcf58af34c,6554d188962337e2abe86f6f046fa280,14:27:22,f29f1d38f73cfc2b28608b50023e487c,85c3fc1ae85ea4526f648fbff2f50f60,23,1.000,226.390,295.790,7e35e74e610188414ad24235dd787c78,295.790,226.390,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,4fc848051e4459b8a6afeb210c3664ec,70e36a157c96b7187e8efe75f168f9b9,Facades,F215,2020-12-17,2020-12-23,test,7 days,2020-12-10
34329950,0132c6d2169585f82c568e79a66368b2,2020-09-13,e90b327c1ceca11f1a44a8bcf58af34c,6554d188962337e2abe86f6f046fa280,14:27:22,f29f1d38f73cfc2b28608b50023e487c,85c3fc1ae85ea4526f648fbff2f50f60,23,1.000,226.390,295.790,7e35e74e610188414ad24235dd787c78,295.790,226.390,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,4fc848051e4459b8a6afeb210c3664ec,70e36a157c96b7187e8efe75f168f9b9,Seasonal,432,2020-10-27,2020-11-16,test,21 days,2020-10-06
34329951,21639bc97bd40da064119925cd56c3ce,2020-07-15,fcd04e26e900e94b9ed6dd604fed2b64,4b5b734b13a7266445cd3fd7786ab584,17:45:16,f29f1d38f73cfc2b28608b50023e487c,no_offers,12,1.000,226.390,312.000,152f1b77a32508570e2745daf9ce7aec,312.000,226.390,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,4fc848051e4459b8a6afeb210c3664ec,70e36a157c96b7187e8efe75f168f9b9,Facades,F215,2020-12-17,2020

In [52]:
checks_with_offers_test_pivot = checks_with_offers_test.pivot_table(index=['Offer_ID', 'sku', 'Promo_type'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_offers_test_pivot.columns = ['selling_price_per_unit', 'supplier_price_per_unit']
checks_with_offers_test_pivot = checks_with_offers_test_pivot.reset_index()
checks_with_offers_test_pivot

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297
...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718


In [53]:
checks_with_offers_test_pivot_amount= checks_with_offers_test.pivot_table(index=['Offer_ID', 'sku', 'Promo_type'], values=['num_sales'] , aggfunc=['sum'])
checks_with_offers_test_pivot_amount.columns = ['num_sales']
checks_with_offers_test_pivot_amount = checks_with_offers_test_pivot_amount.reset_index()
checks_with_offers_test_pivot_amount

,Offer_ID,sku,Promo_type,num_sales
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,82.000
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,165.000
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,1454.000
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,504.000
4,100,752e96b000091940545ce06692bf67c5,Seasonal,514.000
...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,135.000
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,196.000
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,211.000
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,160.000


In [54]:
checks_with_action_test_merged = checks_with_offers_test_pivot.merge(checks_with_offers_test_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'Promo_type'])
checks_with_action_test_merged

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit,num_sales
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005,82.000
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507,165.000
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712,1454.000
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587,504.000
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297,514.000
...,...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879,135.000
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153,196.000
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446,211.000
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718,160.000


In [55]:
checks_with_action_test_merged['margin'] = checks_with_action_test_merged['selling_price_per_unit'] / checks_with_action_test_merged['supplier_price_per_unit'] - 1

In [56]:
checks_with_action_test_merged = checks_with_action_test_merged.merge(hierarchy, how='inner', on = ['sku'])
checks_with_action_test_merged

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit,num_sales,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005,82.000,0.207,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507,165.000,0.066,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,1400ffe327b1b15130fe85535eee58ba
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712,1454.000,-0.061,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587,504.000,-0.081,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297,514.000,-0.057,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
...,...,...,...,...,...,...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879,135.000,0.645,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153,196.000,0.351,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446,211.000,0.386,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718,160.000,0.372,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d


In [48]:
checks_with_offers_test_pivot_merged_offerid = checks_with_action_test_merged.pivot_table(index=['Offer_ID', 'Promo_type'], values=['selling_price_per_unit', 'supplier_price_per_unit', 'margin'] , aggfunc=['mean'])
checks_with_offers_test_pivot_merged_offerid.columns = ['margin', 'selling_price_per_unit', 'supplier_price_per_unit', ]
checks_with_offers_test_pivot_merged_offerid = checks_with_offers_test_pivot_merged_offerid.reset_index()
checks_with_offers_test_pivot_merged_offerid

,Offer_ID,Promo_type,margin,selling_price_per_unit,supplier_price_per_unit
0,100,Seasonal,0.086,222.123,203.559
1,172,Biweekly,0.176,197.775,168.234
2,173,Biweekly,0.752,220.045,128.284
3,174,Biweekly,0.683,100.695,59.943
4,175,Biweekly,0.028,144.794,141.161
...,...,...,...,...,...
144,F216,Facades,0.226,43.723,35.875
145,F217,Facades,0.676,49.172,29.509
146,F218,Facades,0.165,103.323,88.783
147,F219,Facades,0.488,170.549,114.509


In [57]:
test = checks_with_action_test_merged
test

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit,num_sales,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005,82.000,0.207,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507,165.000,0.066,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,1400ffe327b1b15130fe85535eee58ba
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712,1454.000,-0.061,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587,504.000,-0.081,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297,514.000,-0.057,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
...,...,...,...,...,...,...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879,135.000,0.645,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153,196.000,0.351,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446,211.000,0.386,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718,160.000,0.372,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d


## Моделирование

## Модель CatBoost

In [84]:
train.columns

Index(['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'start_date',
       'train_test_group', 'mean_selling_price_per_unit_action',
       'mean_supplier_price_per_unit_action', 'Promo_type_y_x',
       'num_sales_action', 'selling_price_action', 'supplier_price_action',
       'Promo_type_x_y', 'mean_selling_price_per_unit_without_action',
       'mean_supplier_price_per_unit_without_action', 'Promo_type_y_y',
       'num_sales_without_action', 'selling_price_without_action',
       'supplier_price_without_action', 'uplift', 'hierarchy_level1',
       'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4', 'margin'],
      dtype='object')

In [85]:
test.columns

Index(['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit',
       'supplier_price_per_unit', 'num_sales', 'margin', 'hierarchy_level1',
       'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4', 'UpLift',
       'UpSales'],
      dtype='object')

In [86]:
features_train = train[['Offer_ID', 'sku', 'Promo_type_x_x', 'mean_selling_price_per_unit_without_action', 'mean_supplier_price_per_unit_without_action', 'margin', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']]
target_train = train['uplift']
features_test = test[['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit', 'supplier_price_per_unit', 'margin', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']]

In [87]:
features_train.columns = ['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit', 'supplier_price_per_unit', 'margin', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']

In [88]:
cat_feature = ['Offer_ID', 'sku', 'Promo_type', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']

In [89]:
model_cat = CatBoostRegressor(loss_function='MAE', random_seed=12345, iterations=150)
model_cat.fit(features_train, target_train, cat_features=cat_feature, verbose=10)

0:	learn: 0.1436028	total: 4.12ms	remaining: 614ms
10:	learn: 0.1202207	total: 36.3ms	remaining: 459ms
20:	learn: 0.1044996	total: 67.5ms	remaining: 414ms
30:	learn: 0.0943459	total: 99.1ms	remaining: 380ms
40:	learn: 0.0874660	total: 132ms	remaining: 350ms
50:	learn: 0.0823480	total: 164ms	remaining: 317ms
60:	learn: 0.0785250	total: 197ms	remaining: 287ms
70:	learn: 0.0757920	total: 230ms	remaining: 255ms
80:	learn: 0.0738016	total: 261ms	remaining: 223ms
90:	learn: 0.0720870	total: 293ms	remaining: 190ms
100:	learn: 0.0707116	total: 325ms	remaining: 158ms
110:	learn: 0.0695032	total: 357ms	remaining: 125ms
120:	learn: 0.0682345	total: 388ms	remaining: 93ms
130:	learn: 0.0669831	total: 419ms	remaining: 60.8ms
140:	learn: 0.0661291	total: 451ms	remaining: 28.8ms
149:	learn: 0.0653418	total: 479ms	remaining: 0us


In [90]:
predictions_cat_train = model_cat.predict(features_train)
predictions_cat_test = model_cat.predict(features_test)

In [91]:
result_cat = mean_absolute_error(target_train, predictions_cat_train)
print("MAE для catboost:", result_cat)

MAE для catboost: 0.060741253483022575


In [92]:
r2_cat = r2_score(target_train, predictions_cat_train)
r2_cat

0.7025020391951403

In [93]:
test['UpLift'] = predictions_cat_test
test

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit,num_sales,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,UpLift,UpSales
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005,82.000,0.207,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5,-0.205,-16.796
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507,165.000,0.066,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,1400ffe327b1b15130fe85535eee58ba,-0.095,-15.653
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712,1454.000,-0.061,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5,-0.076,-110.261
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587,504.000,-0.081,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5,-0.070,-35.368
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297,514.000,-0.057,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5,-0.071,-36.282
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879,135.000,0.645,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d,-0.201,-27.096
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153,196.000,0.351,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d,-0.156,-30.645
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446,211.000,0.386,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d,-0.154,-32.429
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718,160.000,0.372,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d,-0.170,-27.148


In [76]:
test['UpSales'] = test['UpLift'] * test['num_sales']

In [82]:
test_result_new = test.pivot_table(index=['Offer_ID'], values=['UpSales', 'num_sales'] , aggfunc=['sum'])
test_result_new.columns = ['num_sales', 'UpSales']
test_result_new = test_result_new.reset_index()
test_result_new['UpLift_weight'] = test_result_new['num_sales'] / test_result_new['UpSales']
test_result_new.sort_values(by = 'UpSales')

,Offer_ID,num_sales,UpSales,UpLift_weight
137,F209,-4.661,58.000,-0.080
22,320,-28.250,159.000,-0.178
71,97,-104.639,246.000,-0.425
59,538,-44.465,489.000,-0.091
25,370,-102.415,576.000,-0.178
...,...,...,...,...
106,F175,-22862.525,291490.340,-0.078
61,541,-46931.227,319961.000,-0.147
93,B95,-36753.286,390656.630,-0.094
80,B140,-41419.723,392848.586,-0.105


## Модель LightGMB

In [ ]:
features_train['Offer_ID'] = features_train['Offer_ID'].astype('category')
features_test['Offer_ID'] = features_test['Offer_ID'].astype('category')
features_train['sku'] = features_train['sku'].astype('category')
features_test['sku'] = features_test['sku'].astype('category')
features_train['region_name'] = features_train['region_name'].astype('category')
features_test['region_name'] = features_test['region_name'].astype('category')
features_train['Promo_type_x_x'] = features_train['Promo_type_x_x'].astype('category')
features_test['Promo_type_x_x'] = features_test['Promo_type_x_x'].astype('category')
features_train['hierarchy_level1'] = features_train['hierarchy_level1'].astype('category')
features_test['hierarchy_level1'] = features_test['hierarchy_level1'].astype('category')
features_train['hierarchy_level2'] = features_train['hierarchy_level2'].astype('category')
features_test['hierarchy_level2'] = features_test['hierarchy_level2'].astype('category')
features_train['hierarchy_level3'] = features_train['hierarchy_level3'].astype('category')
features_test['hierarchy_level3'] = features_test['hierarchy_level3'].astype('category')
features_train['hierarchy_level4'] = features_train['hierarchy_level4'].astype('category')
features_test['hierarchy_level4'] = features_test['hierarchy_level4'].astype('category')

In [ ]:
cat_feature = ['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'hierarchy_level1','hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']
model_light = LGBMRegressor(num_leaves = 31, max_depth = -1, random_state=12345, n_estimators=100, categorical_feature = cat_feature)
model_light.fit(features_train, target_train, verbose=10)

In [ ]:
predictions_light_train = model_light.predict(features_train)

In [ ]:
result_light = mean_absolute_error(target_train, predictions_light_train)
print("MAE для lightgmb:", result_light)

In [ ]:
predictions_light_test = model_light.predict(features_test)

In [ ]:
result_light = mean_absolute_error(target_test, predictions_light_test)
print("MAE для lightgmb:", result_light)